# QUIMB opt test
Created 23/05/2024

Objectives:
* Got QUIMB working in [this notebook](first_quimb.ipynb), now vary hyperaparameters to check performance.

# Package imports

In [ ]:
import sys

In [ ]:
sys.path.append("../../")

In [ ]:
from itertools import chain
import re

In [ ]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
import quimb as qu
import quimb.tensor as qtn
from quimb.tensor.optimize import TNOptimizer

# Load data

In [ ]:
DATA_DIR_1 = r"../../data/interpolated_trivial_to_nontrivial_fermionic_trivial_proj_rep_200_site_dmrg/"
DATA_DIR_2 = r"../../data/interpolated_nontrivial_fermionic_proj_rep_to_nontrivial_proj_rep_200_site_dmrg/"

In [ ]:
def parse_file_name(file_name):
    interpolation = int(file_name.split('_')[0])/100

    return interpolation

In [ ]:
loaded_data_non_triv_proj_rep = dict()
energies_non_triv_proj_rep = dict()

for local_file_name in list(os.walk(DATA_DIR_2))[0][2]:
    f_name = r"{}/{}".format(DATA_DIR_2, local_file_name, ignore_unknown=False)

    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data_non_triv_proj_rep[data_info]=data['wavefunction']
        energies_non_triv_proj_rep[data_info]=data['energy']

In [ ]:
psi = loaded_data_non_triv_proj_rep[0.1]

In [ ]:
from collections import Counter

In [ ]:
Counter(
    tuple(psi.get_B(i).get_leg_labels())
    for i in range(psi.L)
)

In [ ]:
psi_arrays = list()
psi_arrays.append(psi.get_B(0, 'Th')[0, ...].to_ndarray())
for i in range(1, psi.L-1):
    psi_arrays.append(psi.get_B(i).to_ndarray())
psi_arrays.append(psi.get_B(psi.L-1)[..., 0].to_ndarray())

In [ ]:
q1 = (
    qtn
    .tensor_1d
    .MatrixProductState(
        psi_arrays,
        shape='lpr'
    )
)

# Definitions

## Define tensor network to contract against

In [ ]:
np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

In [ ]:
q2 = q1.copy(deep=True)

symmetry_sites = list(range(60, 60+80, 2))
for i in symmetry_sites:

    q2.gate(
        np_10,
        where=i,
        contract=False,
        inplace=True
    )

In [ ]:
qb = q1.copy()

In [ ]:
indices_to_map = list(chain(range(60-6,60), range(140, 140+6)))

In [ ]:
indices_to_map

In [ ]:
index_mapping = {f'k{i}': f'b{i}' for i in indices_to_map}

In [ ]:
qb.reindex(index_mapping, inplace=True)

In [ ]:
num_sites = 6
num_symmetry_sites = 80
num_psi_sites = psi.L
left_most_symmetry_site=60

sites_to_contract = {
    'left': list(range(left_most_symmetry_site-num_sites)),
    'middle': list(range(left_most_symmetry_site, left_most_symmetry_site+num_symmetry_sites)),
    'right': list(range(left_most_symmetry_site+num_symmetry_sites+num_sites, num_psi_sites))
}

tags_to_contract = {
    k: [f'I{i}' for i in v]
    for k, v in sites_to_contract.items()
}

In [ ]:
tnc = (
    tn
    .contract(tags_to_contract['left'])
    .contract(tags_to_contract['middle'])
    .contract(tags_to_contract['right'])
)

## Initial mpos

In [ ]:
ml = qtn.MPO_rand(
    6,
    6,
    phys_dim=[4,2],
    normalize=True,
    sites=list(range(54, 60)),
    tags='left_mpo'
)

In [ ]:
mr = qtn.MPO_rand(
    6,
    6,
    phys_dim=[4,2],
    normalize=True,
    sites=list(range(140,146)),
    tags='right_mpo'
)

In [ ]:
mpo = (ml & mr)

## Optimisation functions

In [ ]:
def overlap_loss_function(ml, mr, rdm_tn):
    c = (rdm_tn & ml & mr) ^ ...

    c_abs = abs(c)

    loss = (c_abs - 1)**2

    return loss

In [ ]:
total_physical_dim = 2**9

In [ ]:
regex_s = r"^I\d+$"
regex_p = re.compile(regex_s)

In [ ]:
def relabel_mpo(mpo, k_label, b_label):
    site_locs = [
        int(k[1:]) for k in mpo.tag_map
        if bool(re.search(regex_p, k))
    ]

    k_in_indices = [f'k{i}' for i in site_locs]
    j_in_indices = [f'b{i}' for i in site_locs]

    k_out_indices = [f'{k_label}{i}' for i in site_locs]
    j_out_indices = [f'{b_label}{i}' for i in site_locs]

    mapping = dict(
        chain(
            zip(k_in_indices, k_out_indices),
            zip(j_in_indices, j_out_indices)
        )
    )

    mpo.reindex(mapping, inplace=True)

In [ ]:
def unitarity_tn(tn, total_physical_dim):
    ms = [tn.copy(), tn.copy(), tn.copy()]

    relabel_mpo(ms[0], 'k', 'l')
    relabel_mpo(ms[1], 'm', 'l')
    relabel_mpo(ms[2], 'm', 'b')

    ms[0] = ms[0].conj()
    ms[2] = ms[2].conj()

    n2tn = (tn & tn.conj())
    n2 = n2tn.contract(n2tn.tag_map)
    n4tn = (tn & ms[0] & ms[1] & ms[2])
    n4 = n4tn.contract(n4tn.tag_map)

    return n4 - 2*n2 + total_physical_dim

In [ ]:
unitarity_tn(ml, total_physical_dim)

In [ ]:
def overall_loss_function(mpo, rdm_tn, total_physical_dimension,
    unitary_cost_coefficient=1, overlap_cost_coefficient=1, losses=None):
    ml = qtn.TensorNetwork(
        list(map(mpo.tensor_map.__getitem__, mpo.tag_map['left_mpo']))
    )
    mr = qtn.TensorNetwork(
        list(map(mpo.tensor_map.__getitem__, mpo.tag_map['right_mpo']))
    )
    o_loss = overlap_loss_function(ml, mr, rdm_tn)
    ul_loss = unitarity_tn(ml, total_physical_dimension)
    ur_loss = unitarity_tn(mr, total_physical_dimension)

    out = (
        unitary_cost_coefficient*(ul_loss+ur_loss)
        + overlap_cost_coefficient*o_loss
    )

    if losses is not None:
        losses.append((o_loss, ul_loss, ur_loss))
    return out

# Optimization runs

## 1

In [ ]:
loss_data=list()

In [ ]:
optmzr = qtn.optimize.TNOptimizer(
    mpo,                                # our initial input, the tensors of which to optimize
    loss_fn=overall_loss_function,
    loss_kwargs={
        'rdm_tn': tnc,
        'total_physical_dimension': total_physical_dim,
        'unitary_cost_coefficient': 1,
        'overlap_cost_coefficient': 50,
        'losses': loss_data
    },
    autodiff_backend='tensorflow',      # {'jax', 'tensorflow', 'autograd'}
    optimizer='L-BFGS-B',               # supplied to scipy.minimize
)

In [ ]:
mpo_opt = optmzr.optimize(1000)

In [ ]:
len(loss_data)

In [ ]:
optmzr.plot()

In [ ]:
X = np.array(loss_data)

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

In [ ]:
X = np.array(loss_data)[100:]

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

## 2

In [ ]:
loss_data=list()

In [ ]:
optmzr = qtn.optimize.TNOptimizer(
    mpo,                                # our initial input, the tensors of which to optimize
    loss_fn=overall_loss_function,
    loss_kwargs={
        'rdm_tn': tnc,
        'total_physical_dimension': total_physical_dim,
        'unitary_cost_coefficient': 1,
        'overlap_cost_coefficient': 200,
        'losses': loss_data
    },
    autodiff_backend='tensorflow',      # {'jax', 'tensorflow', 'autograd'}
    optimizer='L-BFGS-B',               # supplied to scipy.minimize
)

In [ ]:
mpo_opt = optmzr.optimize(1000)

In [ ]:
len(loss_data)

In [ ]:
optmzr.plot()

In [ ]:
X = np.array(loss_data)

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

In [ ]:
X = np.array(loss_data)[100:]

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

## 3

In [ ]:
loss_data=list()

In [ ]:
optmzr = qtn.optimize.TNOptimizer(
    mpo,                                # our initial input, the tensors of which to optimize
    loss_fn=overall_loss_function,
    loss_kwargs={
        'rdm_tn': tnc,
        'total_physical_dimension': total_physical_dim,
        'unitary_cost_coefficient': 1,
        'overlap_cost_coefficient': 500,
        'losses': loss_data
    },
    autodiff_backend='tensorflow',      # {'jax', 'tensorflow', 'autograd'}
    optimizer='L-BFGS-B',               # supplied to scipy.minimize
)

In [ ]:
mpo_opt = optmzr.optimize(1000)

In [ ]:
len(loss_data)

In [ ]:
optmzr.plot()

In [ ]:
X = np.array(loss_data)

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

In [ ]:
X = np.array(loss_data)[100:]

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

## 4

In [ ]:
loss_data=list()

In [ ]:
optmzr = qtn.optimize.TNOptimizer(
    mpo,                                # our initial input, the tensors of which to optimize
    loss_fn=overall_loss_function,
    loss_kwargs={
        'rdm_tn': tnc,
        'total_physical_dimension': total_physical_dim,
        'unitary_cost_coefficient': 1,
        'overlap_cost_coefficient': 5000,
        'losses': loss_data
    },
    autodiff_backend='tensorflow',      # {'jax', 'tensorflow', 'autograd'}
    optimizer='L-BFGS-B',               # supplied to scipy.minimize
)

In [ ]:
mpo_opt = optmzr.optimize(1000)

In [ ]:
len(loss_data)

In [ ]:
optmzr.plot()

In [ ]:
X = np.array(loss_data)

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

In [ ]:
X = np.array(loss_data)[100:]

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

## 5

In [ ]:
loss_data=list()

In [ ]:
optmzr = qtn.optimize.TNOptimizer(
    mpo,                                # our initial input, the tensors of which to optimize
    loss_fn=overall_loss_function,
    loss_kwargs={
        'rdm_tn': tnc,
        'total_physical_dimension': total_physical_dim,
        'unitary_cost_coefficient': 1,
        'overlap_cost_coefficient': 5000,
        'losses': loss_data
    },
    autodiff_backend='tensorflow',      # {'jax', 'tensorflow', 'autograd'}
    optimizer='adam',               # supplied to scipy.minimize
)

In [ ]:
mpo_opt = optmzr.optimize(10000)

In [ ]:
len(loss_data)

In [ ]:
optmzr.plot()

In [ ]:
X = np.array(loss_data)

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

In [ ]:
X = np.array(loss_data)[2000:]

fig, [ax1, ax2] = plt.subplots(nrows=2, figsize=(10, 6))

ax1.plot(X[:,1])
ax1.plot(X[:,2])
ax2.plot(X[:,0])

In [ ]:
X[:,0]

## 6

In [ ]:
loss_data=list()

In [ ]:
optmzr = qtn.optimize.TNOptimizer(
    mpo,                                # our initial input, the tensors of which to optimize
    loss_fn=overall_loss_function,
    loss_kwargs={
        'rdm_tn': tnc,
        'total_physical_dimension': total_physical_dim,
        'unitary_cost_coefficient': 0,
        'overlap_cost_coefficient': 1,
        'losses': loss_data
    },
    autodiff_backend='tensorflow',      # {'jax', 'tensorflow', 'autograd'}
    optimizer='L-BFGS-B',               # supplied to scipy.minimize
)

In [ ]:
mpo_opt = optmzr.optimize(1000)

# Conclusion
It looks like the overlap gradient isn't working...!